In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import math
import torch.nn.functional as F
import numpy as np

In [2]:
transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset_path = '~/Desktop/research_data/SARscope/train_data'
test_dataset_path = '~/Desktop/research_data/SARscope/test_data'

dataset_train = ImageFolder(root = train_dataset_path, transform = transform)
dataset_test = ImageFolder(root = test_dataset_path, transform = transform)

batch_size = 32
dataloader_train = DataLoader(dataset_train, batch_size = batch_size, shuffle = True)
dataloader_test = DataLoader(dataset_test, batch_size = batch_size, shuffle = True)


In [3]:
print("shape: ", dataset_train[0][0].shape)
len(dataloader_train)

shape:  torch.Size([3, 640, 640])


148

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
print(dataset_train[0][0][1].sum() == dataset_train[0][0][0].sum())

tensor(True)


In [6]:
import torch.nn as nn

In [7]:
def relu(x):
    _relu = nn.ReLU()
    
    return _relu(x)

def curly_N(v):
    v_min, v_max = v.min(), v.max()
    new_min, new_max = -.25, .25
    v_p = (v - v_min)/(v_max - v_min)*(new_max - new_min) + new_min
    return v_p

def curly_Nprime(v):
    v_min, v_max = v.min(), v.max()
    new_min, new_max = -.25, .25
    v_pw = (v - v_min + 1)/(v_max - v_min + 2)*(new_max - new_min) + new_min
    return v_pw
    # return (w - np.min(w) + 1) / (np.max(w) - np.min(w) + 2)

def f_VHN(x, w):
    relu_x = relu(curly_N(x))
    relu_w = relu(curly_Nprime(w))
    
    return torch.mul(relu_x, relu_w)

In [8]:
# class MyLinearLayer(nn.Module):
#     """ Custom Linear layer but mimics a standard linear layer """
#     def __init__(self, size_in, size_out):
#         super().__init__()
#         self.size_in, self.size_out = size_in, size_out
#         weights = torch.Tensor(size_out, size_in)
#         self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
#         bias = torch.Tensor(size_out)
#         self.bias = nn.Parameter(bias)

#         # initialize weights and biases
#         nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
#         fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weights)
#         bound = 1 / math.sqrt(fan_in)
#         nn.init.uniform_(self.bias, -bound, bound)  # bias init

#     def forward(self, x):
#         w_times_x= torch.mm(x, self.weights.t())
#         return torch.add(w_times_x, self.bias)  # w times x + b
        
class VHNLayer(nn.Module):
    """ Custom VHN layer """
    def __init__(self, channels, img_len, img_width):
        super().__init__()
        self.channels, self.img_len, self.img_width = channels, img_len, img_width
        weights = torch.Tensor(channels, img_len, img_width)
        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.

        # initialize weights and biases
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        
        

    def forward(self, x):
        
        return f_VHN(x, self.weights) # w times x + b

In [9]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.vhn1 = VHNLayer(3, 640, 640)
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 64, 5)
        self.pool = nn.MaxPool2d(3, 3)
        self.fc1 = nn.Linear(28224, 8000)
        self.fc2 = nn.Linear(8000, 84)
        self.fc3 = nn.Linear(84, 16)
        self.fc4 = nn.Linear(16, 1)

    def forward(self, x):
        x = self.vhn1(x)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        # print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.sigmoid(self.fc4(x))
        # print(x.shape)
        return x


net = ConvNet()


In [10]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


In [12]:
for epoch in range(2):  # loop over the dataset multiple times
    print(epoch)
    running_loss = 0.0
    for i, data in enumerate(dataloader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        if i != 147:
            inputs, labels = data
            
            # zero the parameter gradients
            optimizer.zero_grad()
    
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels.reshape(32,1).type(torch.float32))
            loss.backward()
            optimizer.step()
    
            # print statistics
            running_loss += loss.item()
            print(i)
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

print('Finished Training')

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
Finished Training


In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in dataloader_test:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        print((predicted == labels).sum())
        correct += (predicted == labels).sum().item()
        print(
print(f'Accuracy of the network on the test images: {100 * correct // total} %')


tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
tensor(32)
